In [1]:
import pandas as pd
import numpy as np 
import warnings
warnings.filterwarnings('ignore')

# 전처리

#### 1. 엑셀 파일 불러와서 유효하지 않은 데이터 삭제하기

In [2]:
raw_mill= pd.read_excel('rawdata_mill.xlsx', sheet_name= 'RawData')
idx = raw_mill[raw_mill['q11']==5].index # 우선 인덱스를 추출
raw_mill.drop(idx, inplace=True) # 해당 인덱스의 행을 삭제
raw_mill.head()

,NO,q1,q2_1,q3,q4,q5,q6,q7,q8,q8_10,...,q36,q37,q38,q39,q40,q41,q42,q43,q44,q45
0,1,2,38,3,8,1,1,3,1,NaN,...,5,1,2,3,2,1,2,1,2,2
1,2,1,37,3,2,1,3,2,5,NaN,...,1,1,2,3,2,2,1,1,2,3
2,4,2,33,2,4,1,3,1,5,NaN,...,3,1,2,3,2,4,1,1,2,3
3,5,1,26,1,1,1,3,1,5,NaN,...,2,1,2,3,2,1,1,1,2,1
4,6,2,27,1,13,1,3,1,5,NaN,...,3,1,5,3,2,1,1,1,2,2


#### 2. 인구통계학적 변수 전처리

In [3]:
# 인구통계학적 변수만 뽑는다. 
mill_demo = raw_mill[['q1', 'q2_1', 'q3', 'q6', 'q7', 'q8', 'q9', 'q10_1', 'q10_2', 'q10_3', 'q10_4']]
mill_demo.head()

,q1,q2_1,q3,q6,q7,q8,q9,q10_1,q10_2,q10_3,q10_4
0,2,38,3,1,3,1,3,1.0,NaN,NaN,4.0
1,1,37,3,3,2,5,2,NaN,NaN,NaN,NaN
2,2,33,2,3,1,5,2,NaN,NaN,3.0,NaN
3,1,26,1,3,1,5,3,NaN,NaN,NaN,NaN
4,2,27,1,3,1,5,2,NaN,NaN,NaN,NaN


In [4]:
# 변수명(컬럼명)을 바꾼다. 
mill_demo.columns = ['성별', '연령', '연령_그룹', '교육', '혼인', '직업', '소득', '부동산1', '부동산2', '부동산3', '부동산4']
mill_demo.head()

,성별,연령,연령_그룹,교육,혼인,직업,소득,부동산1,부동산2,부동산3,부동산4
0,2,38,3,1,3,1,3,1.0,NaN,NaN,4.0
1,1,37,3,3,2,5,2,NaN,NaN,NaN,NaN
2,2,33,2,3,1,5,2,NaN,NaN,3.0,NaN
3,1,26,1,3,1,5,3,NaN,NaN,NaN,NaN
4,2,27,1,3,1,5,2,NaN,NaN,NaN,NaN


In [5]:
# 중복 응답 문항인 부동산 항목을 '부동산 개수' 항목으로 count한다. 
## 무응답을 0으로 바꿈 
mill_demo.fillna(0, inplace=True)
mill_demo.head()

,성별,연령,연령_그룹,교육,혼인,직업,소득,부동산1,부동산2,부동산3,부동산4
0,2,38,3,1,3,1,3,1.0,0.0,0.0,4.0
1,1,37,3,3,2,5,2,0.0,0.0,0.0,0.0
2,2,33,2,3,1,5,2,0.0,0.0,3.0,0.0
3,1,26,1,3,1,5,3,0.0,0.0,0.0,0.0
4,2,27,1,3,1,5,2,0.0,0.0,0.0,0.0


In [6]:
## 다음으로 0이나 1이 아닌 값을 모두 1로 바꿈
mill_demo.loc[(mill_demo.부동산2 == 2.0) , '부동산2'] = 1.0
mill_demo.loc[(mill_demo.부동산3 == 3.0) , '부동산3'] = 1.0
mill_demo.loc[(mill_demo.부동산4 == 4.0) , '부동산4'] = 1.0

mill_demo.head()

,성별,연령,연령_그룹,교육,혼인,직업,소득,부동산1,부동산2,부동산3,부동산4
0,2,38,3,1,3,1,3,1.0,0.0,0.0,1.0
1,1,37,3,3,2,5,2,0.0,0.0,0.0,0.0
2,2,33,2,3,1,5,2,0.0,0.0,1.0,0.0
3,1,26,1,3,1,5,3,0.0,0.0,0.0,0.0
4,2,27,1,3,1,5,2,0.0,0.0,0.0,0.0


In [7]:
## 부동산1~부동산4의 값을 더한 '부동산 개수' 컬럼을 추가로 생성
mill_demo['부동산 개수'] = mill_demo[['부동산1', '부동산2', '부동산3', '부동산4']].sum(axis=1)
mill_demo = mill_demo.astype({'부동산 개수':'int'})
mill_demo.head(10)

,성별,연령,연령_그룹,교육,혼인,직업,소득,부동산1,부동산2,부동산3,부동산4,부동산 개수
0,2,38,3,1,3,1,3,1.0,0.0,0.0,1.0,2
1,1,37,3,3,2,5,2,0.0,0.0,0.0,0.0,0
2,2,33,2,3,1,5,2,0.0,0.0,1.0,0.0,1
3,1,26,1,3,1,5,3,0.0,0.0,0.0,0.0,0
4,2,27,1,3,1,5,2,0.0,0.0,0.0,0.0,0
5,1,39,3,3,2,5,3,1.0,0.0,0.0,0.0,1
6,1,35,3,2,1,6,2,0.0,0.0,0.0,1.0,1
7,2,38,3,2,2,7,1,1.0,0.0,0.0,0.0,1
8,2,27,1,3,1,5,2,0.0,0.0,0.0,0.0,0
9,1,37,3,3,2,5,3,1.0,0.0,0.0,0.0,1


In [8]:
## 부동산1~부동산4 컬럼 삭제
mill_demo.drop(['부동산1', '부동산2', '부동산3', '부동산4'], axis=1, inplace=True)
mill_demo.head()

,성별,연령,연령_그룹,교육,혼인,직업,소득,부동산 개수
0,2,38,3,1,3,1,3,2
1,1,37,3,3,2,5,2,0
2,2,33,2,3,1,5,2,1
3,1,26,1,3,1,5,3,0
4,2,27,1,3,1,5,2,0


In [9]:
## 응답 수가 부족한 문항을 통합하기

### 교육: 3(대학교 졸업), 4(대학원 재학 이상)을 3(대학교 졸업 이상)으로 통합
### 혼인: 3(이혼, 사별)을 1(미혼) 으로 통합
### 직업: 1,2,3 을 1(자영업)으로 통합 / 4, 5, 6을 2(급여 근로자)로 통합 / 7,8,9,10 3(기타_무직학생주부)로 통합
### 소득: 4, 5 를 3(월400만원 이상)으로 통합

def change(x):
    if x < 4:
        return 1
    elif x < 7:
        return 2
    else:
        return 3

def change2(x):
    if x >= 4:
        return 3
    else:
        return x

mill_demo.직업 = mill_demo['직업'].apply(change)
mill_demo.소득 = mill_demo['소득'].apply(change2)
mill_demo.loc[(mill_demo.교육 == 4) , '교육'] = 3
mill_demo.loc[(mill_demo.혼인 == 3) , '혼인'] = 1
mill_demo.head(5)



,성별,연령,연령_그룹,교육,혼인,직업,소득,부동산 개수
0,2,38,3,1,1,1,3,2
1,1,37,3,3,2,2,2,0
2,2,33,2,3,1,2,2,1
3,1,26,1,3,1,2,3,0
4,2,27,1,3,1,2,2,0


#### 3. 금융투자행동 변수 전처리
#### 최종컬럼 : ['금융상품선택방법', '상품정보획득경로', '금융상품특성', '상품평균보유기간', '보유상품그룹', '투자포트폴리오유형', '대출경험', '투자지속의사']

In [10]:
# 투자행동 관련 변수만 가져온다.
mill_inv = raw_mill[['q11', 'q12', 'q13', 'q14_1', 'q14_2', 'q14_3', 'q14_4', 'q14_5', 'q14_6', 'q14_7', 'q15', 'q16', 'q17', 'q18_1', 'q18_2', 'q18_3', 'q18_4', 'q18_5', 'q18_6', 'q18_7']]

# 결측치는 0으로 채워넣고 정수로 모두 바꾼다
mill_inv.fillna(0, inplace=True)
mill_inv = mill_inv.astype('int')
mill_inv.head()

,q11,q12,q13,q14_1,q14_2,q14_3,q14_4,q14_5,q14_6,q14_7,q15,q16,q17,q18_1,q18_2,q18_3,q18_4,q18_5,q18_6,q18_7
0,1,5,2,1,2,3,4,0,0,0,3,2,1,50,25,5,20,0,0,0
1,4,4,1,1,0,0,0,0,0,0,2,2,1,100,0,0,0,0,0,0
2,1,1,1,1,0,0,4,0,6,0,3,2,1,80,0,0,5,0,15,0
3,1,1,2,1,0,3,0,0,0,0,3,1,1,70,0,30,0,0,0,0
4,1,1,2,1,0,3,4,0,0,0,2,2,1,50,0,20,30,0,0,0


In [11]:
# 1(금융상품그룹1(중저위험)), 2(금융상품그룹2(고위험이상)) 인 '금융상품그룹' 컬럼 생성

def inv(*args):
    if sum(args) > 0:
        return 2
    else:
        return 1

# 컬럼 생성
mill_inv['금융상품그룹'] = mill_inv[['q14_4','q14_5','q14_6','q14_7']].sum(axis=1).apply(inv)


# q14_1 ~7 컬럼 삭제
mill_inv.drop(['q14_1', 'q14_2', 'q14_3', 'q14_4','q14_5','q14_6','q14_7'], axis=1, inplace=True)

mill_inv.head(20)


,q11,q12,q13,q15,q16,q17,q18_1,q18_2,q18_3,q18_4,q18_5,q18_6,q18_7,금융상품그룹
0,1,5,2,3,2,1,50,25,5,20,0,0,0,2
1,4,4,1,2,2,1,100,0,0,0,0,0,0,1
2,1,1,1,3,2,1,80,0,0,5,0,15,0,2
3,1,1,2,3,1,1,70,0,30,0,0,0,0,1
4,1,1,2,2,2,1,50,0,20,30,0,0,0,2
5,1,5,1,5,1,1,10,10,50,30,0,0,0,2
6,1,2,2,3,2,1,50,0,0,50,0,0,0,2
7,3,6,1,5,2,1,50,50,0,0,0,0,0,1
8,1,1,2,3,2,1,40,0,0,30,30,0,0,2
9,1,1,2,3,1,1,100,0,0,0,0,0,0,1


In [12]:
# '포트폴리오 유형1', '포트폴리오 유형2', 포트폴리오 유형3' 컬럼 생성
## 유형1: 1(집중투자), 2(분산투자)
## 유형2: 1(중저위험), 2(고위험이상)
## 유형3: 1(중저위험-집중투자), 2(중저위험-분산투자), 3(고위험이상-집중투자), 4(고위험이상-분산투자)

# 우선 q18_1 ~ 3을 합친 항('중저위험')과 q18_4 ~ 7(고위험이상)을 합친 항('고위험이상') 생성
mill_inv['중저위험'] = mill_inv[['q18_1', 'q18_2', 'q18_3']].sum(axis=1)
mill_inv['고위험이상'] = mill_inv[['q18_4', 'q18_5', 'q18_6', 'q18_7' ]].sum(axis=1)
mill_inv.head(20)



,q11,q12,q13,q15,q16,q17,q18_1,q18_2,q18_3,q18_4,q18_5,q18_6,q18_7,금융상품그룹,중저위험,고위험이상
0,1,5,2,3,2,1,50,25,5,20,0,0,0,2,80,20
1,4,4,1,2,2,1,100,0,0,0,0,0,0,1,100,0
2,1,1,1,3,2,1,80,0,0,5,0,15,0,2,80,20
3,1,1,2,3,1,1,70,0,30,0,0,0,0,1,100,0
4,1,1,2,2,2,1,50,0,20,30,0,0,0,2,70,30
5,1,5,1,5,1,1,10,10,50,30,0,0,0,2,70,30
6,1,2,2,3,2,1,50,0,0,50,0,0,0,2,50,50
7,3,6,1,5,2,1,50,50,0,0,0,0,0,1,100,0
8,1,1,2,3,2,1,40,0,0,30,30,0,0,2,40,60
9,1,1,2,3,1,1,100,0,0,0,0,0,0,1,100,0


In [13]:
# '포트폴리오 유형1' 컬럼 생성

## 먼저 '중저위험' '고위험이상' 컬럼을 리스트로 바꾼다
mid_list = mill_inv['중저위험'].values.tolist()
high_list = mill_inv['고위험이상'].values.tolist()

## total_list 리스트에 넣는다
total_list = []
for i in range(len(mid_list)):
    if mid_list[i] >= 75 or high_list[i] >=75:
        total_list.append(1)
    else:
        total_list.append(2)


# 리스트를 다시 mill_inv 에 새로운 칼럼으로 추가한다. 
mill_inv['포트폴리오 유형1'] = np.array(total_list)
mill_inv.head(10)


# 리스트로 변환해서 조건문 적용 후 다시 컬럼 추가하지 않고 바로 컬럼으로 만들 수 있는 방법?

,q11,q12,q13,q15,q16,q17,q18_1,q18_2,q18_3,q18_4,q18_5,q18_6,q18_7,금융상품그룹,중저위험,고위험이상,포트폴리오 유형1
0,1,5,2,3,2,1,50,25,5,20,0,0,0,2,80,20,1
1,4,4,1,2,2,1,100,0,0,0,0,0,0,1,100,0,1
2,1,1,1,3,2,1,80,0,0,5,0,15,0,2,80,20,1
3,1,1,2,3,1,1,70,0,30,0,0,0,0,1,100,0,1
4,1,1,2,2,2,1,50,0,20,30,0,0,0,2,70,30,2
5,1,5,1,5,1,1,10,10,50,30,0,0,0,2,70,30,2
6,1,2,2,3,2,1,50,0,0,50,0,0,0,2,50,50,2
7,3,6,1,5,2,1,50,50,0,0,0,0,0,1,100,0,1
8,1,1,2,3,2,1,40,0,0,30,30,0,0,2,40,60,2
9,1,1,2,3,1,1,100,0,0,0,0,0,0,1,100,0,1


In [14]:
# '포트폴리오 유형2' 컬럼 생성

## total_list2 리스트에 넣는다
total_list2 = []
for i in range(len(mid_list)):
    if mid_list[i] > high_list[i]:
        total_list2.append(1)
    else:
        total_list2.append(2)

## '포트폴리오 유형2'을 mill_inv 컬럼에 추가한다. 
mill_inv['포트폴리오 유형2'] = np.array(total_list2)
mill_inv.head(10)

,q11,q12,q13,q15,q16,q17,q18_1,q18_2,q18_3,q18_4,q18_5,q18_6,q18_7,금융상품그룹,중저위험,고위험이상,포트폴리오 유형1,포트폴리오 유형2
0,1,5,2,3,2,1,50,25,5,20,0,0,0,2,80,20,1,1
1,4,4,1,2,2,1,100,0,0,0,0,0,0,1,100,0,1,1
2,1,1,1,3,2,1,80,0,0,5,0,15,0,2,80,20,1,1
3,1,1,2,3,1,1,70,0,30,0,0,0,0,1,100,0,1,1
4,1,1,2,2,2,1,50,0,20,30,0,0,0,2,70,30,2,1
5,1,5,1,5,1,1,10,10,50,30,0,0,0,2,70,30,2,1
6,1,2,2,3,2,1,50,0,0,50,0,0,0,2,50,50,2,2
7,3,6,1,5,2,1,50,50,0,0,0,0,0,1,100,0,1,1
8,1,1,2,3,2,1,40,0,0,30,30,0,0,2,40,60,2,2
9,1,1,2,3,1,1,100,0,0,0,0,0,0,1,100,0,1,1


In [15]:
# '포트폴리오 유형3' 컬럼 생성

## 먼저 '포트폴리오 유형1' '포트폴리오 유형2' 컬럼을 리스트로 바꾼다
port1_list = mill_inv['포트폴리오 유형1'].values.tolist()
port2_list = mill_inv['포트폴리오 유형2'].values.tolist()

## total_list3 리스트에 넣는다
total_list3 = []
for i in range(len(port1_list)):
    if port1_list[i] == 1 and port2_list[i] == 1:
        total_list3.append(1)
    elif port1_list[i] == 1 and port2_list[i] == 2:
        total_list3.append(3)
    elif port1_list[i] == 2 and port2_list[i] == 1:
        total_list3.append(2)
    else:
        total_list3.append(4)     

## '포트폴리오 유형3'을 mill_inv 컬럼에 추가한다. 
mill_inv['포트폴리오 유형3'] = np.array(total_list3)

mill_inv

,q11,q12,q13,q15,q16,q17,q18_1,q18_2,q18_3,q18_4,q18_5,q18_6,q18_7,금융상품그룹,중저위험,고위험이상,포트폴리오 유형1,포트폴리오 유형2,포트폴리오 유형3
0,1,5,2,3,2,1,50,25,5,20,0,0,0,2,80,20,1,1,1
1,4,4,1,2,2,1,100,0,0,0,0,0,0,1,100,0,1,1,1
2,1,1,1,3,2,1,80,0,0,5,0,15,0,2,80,20,1,1,1
3,1,1,2,3,1,1,70,0,30,0,0,0,0,1,100,0,1,1,1
4,1,1,2,2,2,1,50,0,20,30,0,0,0,2,70,30,2,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
225,1,1,2,3,2,1,40,5,15,40,0,0,0,2,60,40,2,1,2
226,1,4,2,3,2,3,40,20,40,0,0,0,0,1,100,0,1,1,1
228,1,5,1,3,2,1,20,0,0,80,0,0,0,2,20,80,1,2,3
229,1,4,1,4,2,1,50,0,0,50,0,0,0,2,50,50,2,2,4


In [16]:
# q18_1 ~ q18_7 컬럼, 중저위험, 고위험이상 컬럼을 삭제한다. 
mill_inv.drop(['q18_1', 'q18_2', 'q18_3', 'q18_4', 'q18_5', 'q18_6', 'q18_7', '중저위험', '고위험이상'], axis=1, inplace=True)

# 나머지 컬럼도 이름을 바꿔준다. '금융상품선택방법', '상품정보획득경로', '금융상품특성', '상품평균보유기간', '보유상품그룹', '투자포트폴리오유형', '대출경험', '투자지속의사'
 
mill_inv = mill_inv.rename(columns= {'q11':'금융상품선택방법', 'q12':'상품정보획득경로', 'q13':'금융상품특성', 'q15':'상품평균보유기간', 'q16':'대출경험', 'q17':'투자지속의사'})
mill_inv

,금융상품선택방법,상품정보획득경로,금융상품특성,상품평균보유기간,대출경험,투자지속의사,금융상품그룹,포트폴리오 유형1,포트폴리오 유형2,포트폴리오 유형3
0,1,5,2,3,2,1,2,1,1,1
1,4,4,1,2,2,1,1,1,1,1
2,1,1,1,3,2,1,2,1,1,1
3,1,1,2,3,1,1,1,1,1,1
4,1,1,2,2,2,1,2,2,1,2
...,...,...,...,...,...,...,...,...,...,...
225,1,1,2,3,2,1,2,2,1,2
226,1,4,2,3,2,3,1,1,1,1
228,1,5,1,3,2,1,2,1,2,3
229,1,4,1,4,2,1,2,2,2,4


In [17]:
# 응답 값이 적어 분석에 사용이 어려운 항목들을 통합한다
## 금융상품선택방법: 1,2,3 을 1(다양한 금융상품 또는 회사를 살펴본 후 선택) / 4를 2(단일 상품과 회사만 살펴본 후 선택)로 변경
## 상품정보획득경로: 2, 5 를 2(전문가 제공한 정보)로 통합 / 3,4,6 을 3(기타)으로 통합
## 금융상품특성: 1,3,4 을 1(안전성)으로 통합
## 상품평균보유기간: 1,2를 1(1년 미만)으로 통합
## 투자지속의사: 2,3을 2(아니다 또는 잘 모르겠음) 으로 통합

def func1(x):
    if x == 1 or x == 2 or x == 3:
        return 1
    else:
        return 2

def func2(x):
    if x==2 or x==5:
        return 2
    elif x==1:
        return 1
    else:
        return 3

def func3(x):
    if x == 1 or x == 3 or x == 4:
        return 1
    else:
        return 2

def func4(x):
    if x==1 or x==2:
        return 1
    elif x==3:
        return 2
    elif x==4:
        return 3
    else:
        return 4

    
mill_inv.금융상품선택방법 = mill_inv['금융상품선택방법'].apply(func1)
mill_inv.상품정보획득경로 = mill_inv['상품정보획득경로'].apply(func2)
mill_inv.금융상품특성 = mill_inv['금융상품특성'].apply(func3)
mill_inv.상품평균보유기간 = mill_inv['상품평균보유기간'].apply(func4)
mill_inv.loc[(mill_inv.투자지속의사 == 3) , '투자지속의사'] = 2

mill_inv.head(20)

,금융상품선택방법,상품정보획득경로,금융상품특성,상품평균보유기간,대출경험,투자지속의사,금융상품그룹,포트폴리오 유형1,포트폴리오 유형2,포트폴리오 유형3
0,1,2,2,2,2,1,2,1,1,1
1,2,3,1,1,2,1,1,1,1,1
2,1,1,1,2,2,1,2,1,1,1
3,1,1,2,2,1,1,1,1,1,1
4,1,1,2,1,2,1,2,2,1,2
5,1,2,1,4,1,1,2,2,1,2
6,1,2,2,2,2,1,2,2,2,4
7,1,3,1,4,2,1,1,1,1,1
8,1,1,2,2,2,1,2,2,2,4
9,1,1,2,2,1,1,1,1,1,1


#### 4. 금융 지식 데이터 전처리

In [18]:
# 금융 지식 관련 컬럼만 추출하여 데이터프레임 생성
fin_know = raw_mill.loc[:,'q32':'q45']
fin_know.head()

,q32,q33,q34,q35,q36,q37,q38,q39,q40,q41,q42,q43,q44,q45
0,3,4,1,1,5,1,2,3,2,1,2,1,2,2
1,3,4,1,1,1,1,2,3,2,2,1,1,2,3
2,3,3,1,3,3,1,2,3,2,4,1,1,2,3
3,3,1,3,1,2,1,2,3,2,1,1,1,2,1
4,3,3,1,1,3,1,5,3,2,1,1,1,2,2


In [19]:
# 객관적 금융지식 측정 문항에 대하여 정답은 1로 오답은 0으로 값을 변경한다. 
  
## 정답이 1인 문항 (주석 한번에 단축키 ctrl + /)
# a = int(input()) 
# def fin1(x):
#     if x == a:
#         return 1
#     else:
#         return 0

# fin_know.q34 = fin_know['q34'].apply(fin1) 
# fin_know.q35 = fin_know['q35'].apply(fin1) 
# fin_know.q37 = fin_know['q37'].apply(fin1) 
# fin_know.q41 = fin_know['q41'].apply(fin1) 
# fin_know.q42 = fin_know['q42'].apply(fin1) 
# fin_know.q43 = fin_know['q43'].apply(fin1) 
# fin_know.q45 = fin_know['q45'].apply(fin1) 

_list = ['q33','q34','q35','q36','q37','q38','q39','q40','q41','q42','q43','q44','q45']
_a = [3, 1, 1, 3, 1, 2, 3, 2, 1, 1, 1, 2, 1]

def func_1(x, y):
    if x == y:
        return 1
    else:
        return 0

for i in range(len(_list)):
    _column = _list[i]
    _b = _a[i]
    fin_know[_column] = fin_know[_column].apply(lambda x : func_1(x, _b))

fin_know.head(20)


,q32,q33,q34,q35,q36,q37,q38,q39,q40,q41,q42,q43,q44,q45
0,3,0,1,1,0,1,1,1,1,1,0,1,1,0
1,3,0,1,1,0,1,1,1,1,0,1,1,1,0
2,3,1,1,0,1,1,1,1,1,0,1,1,1,0
3,3,0,0,1,0,1,1,1,1,1,1,1,1,1
4,3,1,1,1,1,1,0,1,1,1,1,1,1,0
5,3,0,1,0,0,1,0,1,1,0,1,1,1,0
6,4,0,1,1,1,1,1,1,1,0,1,0,1,0
7,4,1,1,1,0,1,0,1,1,1,1,0,0,0
8,3,1,1,1,0,1,1,1,1,1,1,0,1,0
9,3,0,1,1,0,1,1,1,0,1,0,0,1,0


In [20]:
# q33~q45의 합을 나타내는 '금융지식점수합' 칼럼 생성

fin_know['금융지식점수합'] = fin_know[['q33', 'q34', 'q35', 'q36', 'q37', 'q38', 'q39', 'q40', 'q41', 'q42', 'q43', 'q44', 'q45']].sum(axis=1)

# q33~q45 컬럼 삭제
fin_know.drop(['q33', 'q34', 'q35', 'q36', 'q37', 'q38', 'q39', 'q40', 'q41', 'q42', 'q43', 'q44', 'q45'], axis=1, inplace=True)

# 주관적금융지식 컬럼명 변경 (1 상, 2 중상, 3 중, 4 중하, 5 하)
fin_know = fin_know.rename(columns={'q32':'주관적금융지식'})
fin_know.head(20)

,주관적금융지식,금융지식점수합
0,3,9
1,3,9
2,3,10
3,3,10
4,3,11
5,3,7
6,4,9
7,4,8
8,3,10
9,3,7


#### 5. 위험수용성향 변수 전처리

In [21]:
# 위험수용성향 관련 컬럼만 추출하여 데이터프레임 생성
risk_tol = raw_mill.loc[:,'q19':'q31']
risk_tol.head(10)

,q19,q20,q21,q22,q23,q24,q25,q26,q27,q28,q29,q30,q31
0,2,4,4,2,2,2,2,4,1,2,3,2,3
1,3,1,2,1,1,1,1,1,1,1,1,1,2
2,2,2,3,2,2,1,1,2,1,2,2,1,2
3,3,2,2,2,2,1,2,3,1,2,4,1,2
4,2,2,3,2,2,3,2,2,1,2,2,2,2
5,2,2,2,2,2,1,2,2,1,2,2,1,2
6,3,1,3,2,2,1,1,3,1,2,2,2,3
7,3,1,3,1,1,1,2,1,1,1,1,1,1
8,3,1,3,3,2,2,3,4,1,1,3,1,2
9,2,2,3,2,2,1,2,2,1,2,1,2,2


In [22]:
# 점수 값 할당하여 변경
def risk(x):
    if x == 2:
        return 3
    else:
        return 1

risk_tol.q27 = risk_tol['q27'].apply(risk)
risk_tol.q28 = risk_tol['q28'].apply(risk)

risk_tol.head(20)

,q19,q20,q21,q22,q23,q24,q25,q26,q27,q28,q29,q30,q31
0,2,4,4,2,2,2,2,4,1,3,3,2,3
1,3,1,2,1,1,1,1,1,1,1,1,1,2
2,2,2,3,2,2,1,1,2,1,3,2,1,2
3,3,2,2,2,2,1,2,3,1,3,4,1,2
4,2,2,3,2,2,3,2,2,1,3,2,2,2
5,2,2,2,2,2,1,2,2,1,3,2,1,2
6,3,1,3,2,2,1,1,3,1,3,2,2,3
7,3,1,3,1,1,1,2,1,1,1,1,1,1
8,3,1,3,3,2,2,3,4,1,1,3,1,2
9,2,2,3,2,2,1,2,2,1,3,1,2,2


In [23]:
# q19~31 합을 할당한 '위험수용성향점수합' 컬럼 생성

risk_tol['위험수용성향점수합'] = risk_tol[['q19', 'q20', 'q21', 'q22', 'q23', 'q24', 'q25', 'q26', 'q27', 'q28', 'q29', 'q30', 'q31']].sum(axis=1)
risk_tol.drop(['q19', 'q20', 'q21', 'q22', 'q23', 'q24', 'q25', 'q26', 'q27', 'q28', 'q29', 'q30', 'q31'], axis=1, inplace=True)

risk_tol.head()

,위험수용성향점수합
0,34
1,17
2,24
3,28
4,28


#### 6. 인구통계학적 변수, 금융자산투자행동 변수, 금융지식 변수, 위험수용성향 점수 데이터프레임을 통합

In [24]:
pd.set_option('display.max_columns', None)
finish_millennial = pd.concat([mill_demo, mill_inv, fin_know, risk_tol],axis=1)
finish_millennial.head(10)

,성별,연령,연령_그룹,교육,혼인,직업,소득,부동산 개수,금융상품선택방법,상품정보획득경로,금융상품특성,상품평균보유기간,대출경험,투자지속의사,금융상품그룹,포트폴리오 유형1,포트폴리오 유형2,포트폴리오 유형3,주관적금융지식,금융지식점수합,위험수용성향점수합
0,2,38,3,1,1,1,3,2,1,2,2,2,2,1,2,1,1,1,3,9,34
1,1,37,3,3,2,2,2,0,2,3,1,1,2,1,1,1,1,1,3,9,17
2,2,33,2,3,1,2,2,1,1,1,1,2,2,1,2,1,1,1,3,10,24
3,1,26,1,3,1,2,3,0,1,1,2,2,1,1,1,1,1,1,3,10,28
4,2,27,1,3,1,2,2,0,1,1,2,1,2,1,2,2,1,2,3,11,28
5,1,39,3,3,2,2,3,1,1,2,1,4,1,1,2,2,1,2,3,7,24
6,1,35,3,2,1,2,2,1,1,2,2,2,2,1,2,2,2,4,4,9,27
7,2,38,3,2,2,3,1,1,1,3,1,4,2,1,1,1,1,1,4,8,18
8,2,27,1,3,1,2,2,0,1,1,2,2,2,1,2,2,2,4,3,10,29
9,1,37,3,3,2,2,3,1,1,1,2,2,1,1,1,1,1,1,3,7,25


#### 7. 엑셀 파일로 저장

In [25]:
finish_millennial.to_excel(excel_writer='millennial_fin_pre.xlsx', index=False)